In [1]:
import numpy as np
import requests
from elasticsearch import Elasticsearch, helpers
import time
es = Elasticsearch("192.168.2.104:9200")


In [130]:
def readData():
    dataAll = []
    for i in range(9):
        url = "/home/test/下载/vectors/myfeature_" + str(i) +".npy"
        print(url)
        imgData=np.load('/home/test/下载/vectors/myfeature_0.npy',encoding = "latin1")  #加载文件
        dataAll.append(list(imgData))
    return dataAll
res = readData()    

/home/test/下载/vectors/myfeature_0.npy
/home/test/下载/vectors/myfeature_1.npy
/home/test/下载/vectors/myfeature_2.npy
/home/test/下载/vectors/myfeature_3.npy
/home/test/下载/vectors/myfeature_4.npy
/home/test/下载/vectors/myfeature_5.npy
/home/test/下载/vectors/myfeature_6.npy
/home/test/下载/vectors/myfeature_7.npy
/home/test/下载/vectors/myfeature_8.npy


In [131]:
img_data = res[0] + res[1] + res[2]
label_data  = res[3] + res[4] + res[5]
recom_data = res[6] + res[7] + res[8]
print(len(img_data),len(label_data),len(recom_data))

30000 30000 30000


In [136]:
actions = []
s=time.time()
for i in range(30000):
    action = {
        "_index":"test_vector",
        "_type":"_doc",
        "_source":{
            "label_vector":label_data[i].tolist(),
            "image_vector":img_data[i].tolist(),
            "recom_vector":recom_data[i].tolist(),
            "status":"published"
        }
    }
    actions.append(action)
print(len(actions))
a = helpers.bulk(es,actions)
# e = time.time()

30000


In [ ]:
start = time.time()
for i in range(2000000):
    test1=np.random.uniform(-1,1,size=32).tolist()
    test2=np.random.uniform(-1,1,size=32).tolist()
    test3=np.random.uniform(-1,1,size=32).tolist()
    bodys = {
        "image_vector":test1,
        "recom_vector":test2,
        "label_vector":test3,
        "status":"published"
    }
    est = es.index(index="test_lp",doc_type="_doc",body=bodys)
print('写完了',time.time()-start)

In [159]:
x = [1 for i in range(32)]
print(x)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [163]:
# test = img_data[0].tolist()
test=np.random.uniform(-1,1,size=32).tolist()
print(test)

[0.19079836293547903, -0.5584705331471804, -0.643141124678263, -0.9149045345495816, -0.26212321020522933, -0.5749009705402328, -0.22479785595771662, 0.963007977842675, 0.09335752122879004, 0.42026909765474074, 0.1777296462112654, 0.9402159197403368, -0.008646319722716012, -0.9794435285195517, -0.9208209371405838, -0.761204874236715, 0.5803233936983783, 0.9105254999459811, -0.06280966549736777, 0.45202716014365074, 0.957291310502407, -0.8054621745877906, 0.48737309808780127, 0.1864628542143587, -0.4578361636332542, 0.3576410092567732, 0.8131301209918016, -0.7783249240662264, -0.8226432362917051, -0.8262774582662247, -0.8231462102073233, 0.10914760298669646]


In [161]:
test_body = {
    "size":10,
    "query": {
        "script_score": {
            "query": {
                "bool": {
                    "filter": {
                        "term": {
                            "status": "published"
                        }
                    }
                }
            },
            "script": {
                "source": "1 / (1 + l2norm(params.queryVector, doc['image_vector']))",
                "params": {
                    "queryVector": test
                }
            }
        }
    }
}
data = es.search(index="test_vector",body=test_body)

In [162]:
print(data["hits"]["hits"][0]['_source'])

{'label_vector': [0.12495578825473785, 0.18021655082702637, -0.40119728446006775, 0.026798157021403313, 0.16425609588623047, -0.04214135557413101, -0.055218592286109924, -0.2023204118013382, 0.03272172063589096, 0.5406187176704407, 0.15762780606746674, -0.3108941316604614, -0.136821910738945, -0.3501245379447937, -0.12882931530475616, -0.4889468848705292, -0.29470717906951904, 0.24864548444747925, -0.18955744802951813, 0.09597384929656982, 0.07167097181081772, 0.2703656852245331, 0.09742337465286255, 0.4692767858505249, -0.08252671360969543, 0.05014169216156006, -0.3841765224933624, 0.3675885498523712, -0.6570165157318115, 0.11452966928482056, 0.00022129714488983154, -0.05986358970403671], 'image_vector': [0.12495578825473785, 0.18021655082702637, -0.40119728446006775, 0.026798157021403313, 0.16425609588623047, -0.04214135557413101, -0.055218592286109924, -0.2023204118013382, 0.03272172063589096, 0.5406187176704407, 0.15762780606746674, -0.3108941316604614, -0.136821910738945, -0.35012

In [95]:
print('数据',type(list(img_data)))

数据 <class 'list'>


In [169]:
test=np.random.uniform(-1,1,size=32).tolist()
index_name = ["label_vector", "image_vector", "recom_vector"]
results = []
for index in index_name:
    body = {
        "size": 2,
        "query": {
            "script_score": {
                "query": {
                    "bool": {
                        "filter": {
                            "term": {
                                "status": "published"
                            }
                        }
                    }
                },
                "script": {
           "source": "1 / (1 + l2norm(params.queryVector, doc[\'" + index + "\']))",
                    "params": {
                        "queryVector": test
                    }
                }
            }
        },
        "_source": [index]
    }
    s = es.search(index="test_vector", body=body)
    results.append(s)

In [173]:
print(results)

[{'took': 642, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 0.27150494, 'hits': [{'_index': 'test_vector', '_type': '_doc', '_id': '-Ff5_m4Bay-B9ZH4-mnB', '_score': 0.27150494, '_source': {'label_vector': [0.9691442187477519, -0.5514106618552823, 0.14575461148947144, -0.19970886030084234, 0.4682247674514908, 0.5692465322095646, -0.9398775694595072, -0.46027966388656205, -0.21681522842765233, 0.05992512168794217, 0.8997415294720443, 0.23648068137643885, 0.5124160309002026, 0.17069550828793023, -0.5093112429002724, 0.7447736467081862, 0.7852215785880707, -0.20034193537570166, 0.43025045082189384, 0.13896286857766227, 0.5619591319400548, 0.7344021075075082, -0.08442766796732393, -0.5820656288740576, 0.302971671955911, 0.02447990695773461, -0.9059541873011241, 0.0059700352902456455, -0.7161809113495785, -0.5452386516305896, -0.1727644814836844, -0.5859712124488878]}}, {'_index': 

In [76]:
data = es.search(index="test",body={
    "query":{
        "match_all":{}
    }
})

In [77]:
print(data)

{'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [19]:
url = "http://192.168.2.104:9200/test/_mapping/"
r = requests.get(url)
data  = r.json()
print(data)

{'test': {'mappings': {'dynamic': 'strict', 'properties': {'image_vector': {'type': 'dense_vector', 'dims': 32}, 'label_vector': {'type': 'dense_vector', 'dims': 32}, 'recom_vector': {'type': 'dense_vector', 'dims': 32}}}}}


In [192]:
test_body ={
    "mappings":{
        "properties":{
            "image_vector":{
                "type":"dense_vector",
                "dims":32
            },
            "label_vector":{
                "type":"dense_vector",
                "dims":32
            },
            "recom_vector":{
                "type":"dense_vector",
                "dims":32
            },
            "status":{
                "type":"keyword"
            }
        }
    }
}
print(test_body)

{'mappings': {'properties': {'image_vector': {'type': 'dense_vector', 'dims': 32}, 'label_vector': {'type': 'dense_vector', 'dims': 32}, 'recom_vector': {'type': 'dense_vector', 'dims': 32}, 'status': {'type': 'keyword'}}}}


In [193]:
#建表
res = es.indices.create(index="test_lp",body=test_body)

In [ ]:
#增
res = es.index(index="test_lp",doc_type="_doc",body=body)
#删
res = es.delete(index='indexName', doc_type='typeName', id='idValue')
#查
res = es.get(index='indexName', doc_type='typeName', id='idValue')
#改
res  =es.update(index='indexName', doc_type='typeName', id='idValue', body={待更新字段})